# EdgesCGALDelaunay3D — Colab
Build + test à 1 000 000 de points.


In [ ]:
%%bash
set -euxo pipefail
apt-get update -qq
DEBIAN_FRONTEND=noninteractive apt-get install -y -qq build-essential cmake libcgal-dev libtbb-dev libtbbmalloc2 libgmp-dev libmpfr-dev


In [ ]:
# Écrit le projet
import os, json
root = "/content/EdgesCGALDelaunay3D"
os.makedirs(root+"/src", exist_ok=True)
os.makedirs(root+"/data", exist_ok=True)
open(root+"/CMakeLists.txt","w").write("cmake_minimum_required(VERSION 3.22)\nproject(EdgesCGALDelaunay3D LANGUAGES CXX)\n\nset(CMAKE_CXX_STANDARD 17)\nset(CMAKE_CXX_STANDARD_REQUIRED ON)\nif(NOT CMAKE_BUILD_TYPE)\n  set(CMAKE_BUILD_TYPE Release CACHE STRING \"Build type\" FORCE)\nendif()\n\nif(POLICY CMP0167)\n  cmake_policy(SET CMP0167 NEW)\nendif()\n\nfind_package(CGAL REQUIRED)\nfind_package(TBB QUIET)\n\nset(USE_TBB OFF)\nset(TBB_MALLOC_TARGET \"\")\nif(TBB_FOUND)\n  if(TARGET TBB::tbb)\n    set(USE_TBB ON)\n  endif()\n  if(TARGET TBB::tbbmalloc)\n    set(TBB_MALLOC_TARGET TBB::tbbmalloc)\n  else()\n    find_library(TBBMALLOC_LIB tbbmalloc)\n    if(TBBMALLOC_LIB)\n      set(TBB_MALLOC_TARGET ${TBBMALLOC_LIB})\n    endif()\n  endif()\n  if(USE_TBB AND TBB_MALLOC_TARGET STREQUAL \"\")\n    message(WARNING \"TBB found but tbbmalloc missing => disabling parallel CGAL path.\")\n    set(USE_TBB OFF)\n  endif()\nendif()\n\nadd_executable(EdgesCGALDelaunay3D src/EdgesCGALDelaunay3D.cpp)\ntarget_link_libraries(EdgesCGALDelaunay3D PRIVATE CGAL::CGAL)\n\nif(USE_TBB)\n  target_link_libraries(EdgesCGALDelaunay3D PRIVATE TBB::tbb ${TBB_MALLOC_TARGET})\n  target_compile_definitions(EdgesCGALDelaunay3D PRIVATE CGAL_LINKED_WITH_TBB)\nendif()\n\nif(CMAKE_CXX_COMPILER_ID MATCHES \"GNU|Clang\")\n  target_compile_options(EdgesCGALDelaunay3D PRIVATE -O3 -DNDEBUG -march=native -Wall -Wextra)\nelseif(MSVC)\n  target_compile_options(EdgesCGALDelaunay3D PRIVATE /O2 /DNDEBUG /W4)\nendif()\n")
open(root+"/src/EdgesCGALDelaunay3D.cpp","w").write("// EdgesCGALDelaunay3D.cpp (clean warnings)\n// Delaunay triangulation 3D edges extractor using CGAL, fast I/O and optional TBB parallelism.\n#include <CGAL/Exact_predicates_inexact_constructions_kernel.h>\n#include <CGAL/Delaunay_triangulation_3.h>\n#include <CGAL/Delaunay_triangulation_cell_base_3.h>\n#include <CGAL/Triangulation_vertex_base_with_info_3.h>\n#include <CGAL/Triangulation_data_structure_3.h>\n\n#include <cstdio>\n#include <cstdlib>\n#include <cstdint>\n#include <cstring>\n#include <vector>\n#include <string>\n#include <fstream>\n#include <iostream>\n#include <limits>\n#include <utility>\n#include <algorithm>\n#include <thread>\n\n#ifdef CGAL_LINKED_WITH_TBB\n  #include <tbb/global_control.h>\n#endif\n\nstruct BBox {\n    double xmin =  std::numeric_limits<double>::infinity();\n    double ymin =  std::numeric_limits<double>::infinity();\n    double zmin =  std::numeric_limits<double>::infinity();\n    double xmax = -std::numeric_limits<double>::infinity();\n    double ymax = -std::numeric_limits<double>::infinity();\n    double zmax = -std::numeric_limits<double>::infinity();\n    void update(double x, double y, double z) {\n        xmin = std::min(xmin, x); ymin = std::min(ymin, y); zmin = std::min(zmin, z);\n        xmax = std::max(xmax, x); ymax = std::max(ymax, y); zmax = std::max(zmax, z);\n    }\n    bool valid() const { return xmin <= xmax && ymin <= ymax && zmin <= zmax; }\n};\n\nstatic inline bool parse_xyz(const char* s, double& x, double& y, double& z) {\n    while (*s == ' ' || *s == '\\t') ++s;\n    if (*s == '\\0' || *s == '\\n' || *s == '\\r' || *s == '#') return false;\n    char* endp;\n    x = std::strtod(s, &endp); if (endp == s) return false; s = endp;\n    y = std::strtod(s, &endp); if (endp == s) return false; s = endp;\n    z = std::strtod(s, &endp); if (endp == s) return false;\n    return true;\n}\n\nint main(int argc, char** argv) {\n    if (argc != 3) {\n        std::fprintf(stderr,\n            \"Usage: %s input.xyz output.edges\\n\"\n            \"Each input line: x y z (floats)\\n\", argv[0]);\n        return 1;\n    }\n    const char* in_path = argv[1];\n    const char* out_path = argv[2];\n\n    using K   = CGAL::Exact_predicates_inexact_constructions_kernel;\n    using Vb0 = CGAL::Triangulation_vertex_base_3<K>;\n    using Vb  = CGAL::Triangulation_vertex_base_with_info_3<uint32_t, K, Vb0>;\n    using Cb  = CGAL::Delaunay_triangulation_cell_base_3<K>;\n    #ifdef CGAL_LINKED_WITH_TBB\n      using Tds = CGAL::Triangulation_data_structure_3<Vb, Cb, CGAL::Parallel_tag>;\n    #else\n      using Tds = CGAL::Triangulation_data_structure_3<Vb, Cb>;\n    #endif\n    using Dt  = CGAL::Delaunay_triangulation_3<K, Tds>;\n    using Point = K::Point_3;\n\n    #ifdef CGAL_LINKED_WITH_TBB\n    {\n        int threads = (int)std::thread::hardware_concurrency();\n        if (const char* env = std::getenv(\"CGAL_NTHREADS\")) {\n            int req = std::atoi(env);\n            if (req > 0) threads = req;\n        }\n        static tbb::global_control gc(tbb::global_control::max_allowed_parallelism, threads);\n        std::fprintf(stderr, \"[info] TBB enabled, using up to %d threads\\n\", threads);\n    }\n    #endif\n\n    std::FILE* fin = std::fopen(in_path, \"rb\");\n    if (!fin) { std::perror(\"fopen(input)\"); return 2; }\n\n    std::vector<std::pair<Point, uint32_t>> pts;\n    pts.reserve(1<<20);\n    BBox bb;\n\n    // bulk read and parse\n    {\n        const size_t BUFSZ = size_t(1) << 20;\n        std::vector<char> buf(BUFSZ);\n        std::string acc; acc.reserve(BUFSZ);\n        while (true) {\n            size_t n = std::fread(buf.data(), 1, BUFSZ, fin);\n            if (n == 0) break;\n            for (size_t i = 0; i < n; ++i) {\n                char c = buf[i];\n                if (c == '\\r') continue;\n                if (c == '\\n') {\n                    double x,y,z;\n                    if (parse_xyz(acc.c_str(), x,y,z)) {\n                        if (std::isfinite(x) && std::isfinite(y) && std::isfinite(z)) {\n                            bb.update(x,y,z);\n                            pts.emplace_back(Point(x,y,z), (uint32_t)pts.size());\n                        }\n                    }\n                    acc.clear();\n                } else acc.push_back(c);\n            }\n        }\n        if (!acc.empty()) {\n            double x,y,z;\n            if (parse_xyz(acc.c_str(), x,y,z)) {\n                if (std::isfinite(x) && std::isfinite(y) && std::isfinite(z)) {\n                    bb.update(x,y,z);\n                    pts.emplace_back(Point(x,y,z), (uint32_t)pts.size());\n                }\n            }\n        }\n    }\n    std::fclose(fin);\n\n    if (pts.size() < 2) {\n        std::fprintf(stderr, \"Input has < 2 valid points. Nothing to do.\\n\");\n        std::FILE* fout = std::fopen(out_path, \"wb\");\n        if (fout) std::fclose(fout);\n        return 0;\n    }\n    std::fprintf(stderr, \"[info] Loaded %zu points\\n\", pts.size());\n\n    // Triangulation\n    #ifdef CGAL_LINKED_WITH_TBB\n    std::unique_ptr<Dt::Lock_data_structure> lock_holder;\n    Dt::Lock_data_structure* lock_ptr = nullptr;\n    if (bb.valid()) {\n        CGAL::Bbox_3 box(bb.xmin, bb.ymin, bb.zmin, bb.xmax, bb.ymax, bb.zmax);\n        lock_holder.reset(new Dt::Lock_data_structure(box, 64));\n        lock_ptr = lock_holder.get();\n    }\n    Dt dt(Dt::Geom_traits(), lock_ptr);\n    #else\n    Dt dt;\n    #endif\n\n    dt.insert(pts.begin(), pts.end());\n    if (!dt.is_valid(false))\n        std::fprintf(stderr, \"[warning] CGAL triangulation reports invalid structure.\\n\");\n\n    // Write edges\n    std::FILE* fout = std::fopen(out_path, \"wb\");\n    if (!fout) { std::perror(\"fopen(output)\"); return 3; }\n    const size_t OUTBUFSZ = size_t(1) << 20;\n    std::vector<char> filebuf(OUTBUFSZ);\n    setvbuf(fout, filebuf.data(), _IOFBF, filebuf.size());\n\n    std::vector<char> outbuf; outbuf.reserve(OUTBUFSZ);\n    auto flush_buf = [&](){ if (!outbuf.empty()) { std::fwrite(outbuf.data(),1,outbuf.size(),fout); outbuf.clear(); } };\n\n    size_t edge_count = 0;\n    for (auto eit = dt.finite_edges_begin(); eit != dt.finite_edges_end(); ++eit) {\n        auto c = eit->first;\n        int i = eit->second, j = eit->third;\n        auto vi = c->vertex(i);\n        auto vj = c->vertex(j);\n        if (dt.is_infinite(vi) || dt.is_infinite(vj)) continue;\n        uint32_t a = vi->info(), b = vj->info();\n        if (a == b) continue;\n        if (a > b) std::swap(a,b);\n        char line[64];\n        int len = std::snprintf(line, sizeof(line), \"%u %u\\n\", a, b);\n        if (outbuf.size() + (size_t)len > OUTBUFSZ) flush_buf();\n        outbuf.insert(outbuf.end(), line, line + len);\n        ++edge_count;\n    }\n    flush_buf();\n    std::fclose(fout);\n    std::fprintf(stderr, \"[info] Wrote %zu edges to %s\\n\", edge_count, out_path);\n    return 0;\n}\n")
open(root+"/README.md","w").write("# EdgesCGALDelaunay3D\n\nExtraction rapide du **1\u2011squelette** de la triangulation de **Delaunay 3D** avec **CGAL**.\n\n- **Entr\u00e9e**: `.xyz` (une ligne: `x y z`)\n- **Sortie**: fichier texte, une ligne par ar\u00eate finie: `i j` (indices 0\u2011based)\n\n## D\u00e9pendances (Ubuntu 22.04/24.04)\n\n```bash\nsudo apt-get update\nsudo apt-get install -y build-essential cmake libcgal-dev libtbb-dev libtbbmalloc2 libgmp-dev libmpfr-dev\n```\n\n## Build\n\n```bash\ncmake -S . -B build -DCMAKE_BUILD_TYPE=Release\ncmake --build build -j\n```\n\n## Utilisation\n\n```bash\n./build/EdgesCGALDelaunay3D data/example.xyz out.edges\n# Parall\u00e9lisme (si TBB + tbbmalloc d\u00e9tect\u00e9s) :\nCGAL_NTHREADS=$(nproc) ./build/EdgesCGALDelaunay3D data/example.xyz out.edges\n```\n\n## Notes\n- Lecture/\u00e9criture bufferis\u00e9es, insertion **par lot** via `(Point, index)` pour vitesse et faible overhead.\n- Mode parall\u00e8le activ\u00e9 uniquement si `tbb` et `tbbmalloc` sont trouv\u00e9s pour \u00e9viter les erreurs de linkage.\n")
print("Projet écrit dans", root)


In [ ]:
%%bash
set -euxo pipefail
cd /content/EdgesCGALDelaunay3D
cmake -S . -B build -DCMAKE_BUILD_TYPE=Release
cmake --build build -j
ls -lah build/EdgesCGALDelaunay3D


In [ ]:
# Génère 1M points uniformes
import numpy as np
N = 1_000_000
rng = np.random.default_rng(0)
path = "/content/EdgesCGALDelaunay3D/data/test.xyz"
with open(path, 'w') as f:
    CHUNK = 200_000
    remain = N
    while remain>0:
        m = min(CHUNK, remain)
        xyz = rng.random((m,3))
        for i in range(m):
            f.write(f"{xyz[i,0]} {xyz[i,1]} {xyz[i,2]}\n")
        remain -= m
print('Écrit', N, 'points dans', path)


In [ ]:
%%bash
set -euxo pipefail
cd /content/EdgesCGALDelaunay3D
CGAL_NTHREADS=$(nproc)
time ./build/EdgesCGALDelaunay3D data/test.xyz out.edges
echo "Nombre d'arêtes:" $(wc -l < out.edges)
head -n 10 out.edges
